**Analise exploratoria e aplicação algoritimo Regressão linear no dataset "Average car prices - Bazil" do Kaggle**

https://www.kaggle.com/datasets/vagnerbessa/average-car-prices-bazil

In [1]:
#importação das bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import random
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import metrics

In [2]:
df = pd.read_csv('/content/fipe_2022.csv')

In [3]:
#verificando os tipos de dados e presença de NAs
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290275 entries, 0 to 290274
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   year_of_reference   290275 non-null  int64  
 1   month_of_reference  290275 non-null  object 
 2   fipe_code           290275 non-null  object 
 3   authentication      290275 non-null  object 
 4   brand               290275 non-null  object 
 5   model               290275 non-null  object 
 6   fuel                290275 non-null  object 
 7   gear                290275 non-null  object 
 8   engine_size         290275 non-null  float64
 9   year_model          290275 non-null  int64  
 10  avg_price_brl       290275 non-null  float64
 11  age_years           290275 non-null  int64  
dtypes: float64(2), int64(3), object(7)
memory usage: 26.6+ MB


In [4]:
#verifincando nome das colunas
df.columns

Index(['year_of_reference', 'month_of_reference', 'fipe_code',
       'authentication', 'brand', 'model', 'fuel', 'gear', 'engine_size',
       'year_model', 'avg_price_brl', 'age_years'],
      dtype='object')

In [5]:
#alterando nome das colunas
df.rename(columns={'year_of_reference': 'ano_de_referencia', 'month_of_reference': 'mes_de_referencia', 'fipe_code': 'codigo_fipe',
       'authentication': 'autenticacao', 'brand': 'marca', 'model': 'modelo', 'fuel': 'combustível', 'gear': 'cambio', 
       'engine_size': 'potencia_do_motor', 'year_model': 'ano modelo', 'avg_price_brl': 'preco_medio_brl', 'age_years': 'idade_em_anos'}, 
       inplace = True)

In [6]:
#visualização dos dados
df.head()

,ano_de_referencia,mes_de_referencia,codigo_fipe,autenticacao,marca,modelo,combustível,cambio,potencia_do_motor,ano modelo,preco_medio_brl,idade_em_anos
0,2022,January,038001-6,vwmrywl5qs,Acura,NSX 3.0,Gasoline,manual,3.0,1995,43779.0,28
1,2022,January,038001-6,t9mt723qhz,Acura,NSX 3.0,Gasoline,manual,3.0,1994,42244.0,29
2,2022,January,038001-6,tr5wv4z21g,Acura,NSX 3.0,Gasoline,manual,3.0,1993,40841.0,30
3,2022,January,038001-6,s2xxsjz3mt,Acura,NSX 3.0,Gasoline,manual,3.0,1992,39028.0,31
4,2022,January,038001-6,rtm9gj7zk8,Acura,NSX 3.0,Gasoline,manual,3.0,1991,35678.0,32


#Normalização dos dados

In [7]:
df.columns

Index(['ano_de_referencia', 'mes_de_referencia', 'codigo_fipe', 'autenticacao',
       'marca', 'modelo', 'combustível', 'cambio', 'potencia_do_motor',
       'ano modelo', 'preco_medio_brl', 'idade_em_anos'],
      dtype='object')

In [ ]:
#criando código para cada tipo de combustível

In [8]:
df['combustível'].unique()

array(['Gasoline', 'Diesel', 'Alcohol'], dtype=object)

In [9]:
combustivel = []
for i in df['combustível']:
  if i == 'Gasoline':
    combustivel.append(1)
  elif i == 'Diesel':
    combustivel.append(2)
  else:
    combustivel.append(3)

In [10]:
df['combustível'] = combustivel

In [ ]:
#criando código para cada tipo de cambio

In [11]:
df['cambio'].unique()

array(['manual', 'automatic'], dtype=object)

In [12]:
cambio = []
for i in df['cambio']:
  if i == 'manual':
    cambio.append(0)
  else:
    cambio.append(1)

In [13]:
df['cambio'] = cambio

In [ ]:
#criando um código para cada marca / modelo

In [14]:
ds_marca_modelo = df['marca'] + ' / ' + df['modelo']

In [15]:
ds_marca_modelo = pd.DataFrame(ds_marca_modelo.unique())

In [16]:
ds_marca_modelo.shape

(6064, 1)

In [17]:
ds_marca_modelo['cod'] = random.randint(100,7000)

In [18]:
df['cd_marca_modelo'] = df['marca'] + ' / ' + df['modelo']

In [19]:
ds_marca_modelo.head()

,0,cod
0,Acura / NSX 3.0,3566
1,Acura / Legend 3.2/3.5,3566
2,Acura / Integra GS 1.8,3566
3,Agrale / MARRUÁ 2.8 12V 132cv TDI Diesel,3566
4,Agrale / MARRUÁ AM 50 2.8 140cv TDI Diesel,3566


In [21]:
cod_marca = []
for i in df['cd_marca_modelo']:
  for x in ds_marca_modelo[0]:
    if x == i:
      cod_marca.append(int(ds_marca_modelo[ds_marca_modelo[0]==x]['cod'].values))

In [22]:
df['cd_marca_modelo'] = cod_marca

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290275 entries, 0 to 290274
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ano_de_referencia  290275 non-null  int64  
 1   mes_de_referencia  290275 non-null  object 
 2   codigo_fipe        290275 non-null  object 
 3   autenticacao       290275 non-null  object 
 4   marca              290275 non-null  object 
 5   modelo             290275 non-null  object 
 6   combustível        290275 non-null  int64  
 7   cambio             290275 non-null  int64  
 8   potencia_do_motor  290275 non-null  float64
 9   ano modelo         290275 non-null  int64  
 10  preco_medio_brl    290275 non-null  float64
 11  idade_em_anos      290275 non-null  int64  
 12  cd_marca_modelo    290275 non-null  int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 28.8+ MB


In [ ]:
#criando novo dataframe somente com as colunas preditoras e target. E realizando a ordenação das colunas

In [23]:
df.head()

,ano_de_referencia,mes_de_referencia,codigo_fipe,autenticacao,marca,modelo,combustível,cambio,potencia_do_motor,ano modelo,preco_medio_brl,idade_em_anos,cd_marca_modelo
0,2022,January,038001-6,vwmrywl5qs,Acura,NSX 3.0,1,0,3.0,1995,43779.0,28,3566
1,2022,January,038001-6,t9mt723qhz,Acura,NSX 3.0,1,0,3.0,1994,42244.0,29,3566
2,2022,January,038001-6,tr5wv4z21g,Acura,NSX 3.0,1,0,3.0,1993,40841.0,30,3566
3,2022,January,038001-6,s2xxsjz3mt,Acura,NSX 3.0,1,0,3.0,1992,39028.0,31,3566
4,2022,January,038001-6,rtm9gj7zk8,Acura,NSX 3.0,1,0,3.0,1991,35678.0,32,3566


In [25]:
df2 = df[['combustível', 'cambio', 'potencia_do_motor', 'preco_medio_brl', 'idade_em_anos', 'cd_marca_modelo']]

In [26]:
colunas_ordenadas = ['cd_marca_modelo', 'combustível', 'cambio', 'potencia_do_motor', 'idade_em_anos', 'preco_medio_brl']

In [27]:
df2 = df2[colunas_ordenadas]

#Aplicando modelo de Regressão Linear

In [135]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(df2[['cd_marca_modelo','combustível', 'cambio', 'potencia_do_motor',
       'idade_em_anos']], df2['preco_medio_brl'], test_size = 0.2, random_state = 1)
X_treinamento.shape, X_teste.shape

((232220, 5), (58055, 5))

In [136]:
regressao_linear = LinearRegression()

In [137]:
regressao_linear.fit(X_treinamento, y_treinamento)

LinearRegression()

In [138]:
r2_treino = regressao_linear.score(X_treinamento, y_treinamento)

In [139]:
print('R² Treino:', r2_treino)

R² Treino: 0.24803369048674984


In [140]:
print(regressao_linear.coef_, regressao_linear.intercept_)

[     0.         -50355.30261584 -45830.72109279 123867.55857926
 -12863.18790868] 95070.02608468499


In [141]:
prevs = regressao_linear.predict(X_teste)
prevs

array([ -78200.44847053,   10317.41667086, -127271.03985148, ...,
         15081.73717837,  246619.11953457,  167152.98589585])

In [142]:
r2_teste = metrics.r2_score(y_teste,prevs)
print('R² teste:', r2_teste)

R² teste: 0.25124267724779004


In [143]:
treino_ap1_2 = sm.add_constant(X_treinamento)
treino_ap1_2.head()

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,cd_marca_modelo,combustível,cambio,potencia_do_motor,idade_em_anos
91320,3566,1,0,1.6,9
27264,3566,2,0,2.8,23
18671,3566,1,0,1.6,13
212976,3566,1,0,2.0,12
54843,3566,1,0,1.3,19


In [144]:
r1 = sm.OLS(y_treinamento, treino_ap1_2).fit()
r1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        preco_medio_brl   R-squared:                       0.248
Model:                            OLS   Adj. R-squared:                  0.248
Method:                 Least Squares   F-statistic:                 1.915e+04
Date:                Tue, 20 Dec 2022   Prob (F-statistic):               0.00
Time:                        14:58:21   Log-Likelihood:            -3.2395e+06
No. Observations:              232220   AIC:                         6.479e+06
Df Residuals:                  232215   BIC:                         6.479e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
cd_marca_modelo      26.6601      0.631     42.279      0.000      25.424      27.896
combustível       -5.036e+04   1350.502    -37.286      0.000    -5.3e+04   -4.77e+04
cambio            -4.583e+04   1377.505    -33.271      0.000   -4.85e+04   -4.31e+04
potencia_do_motor  1.239e+05    567.651    218.211      0.000    1.23e+05    1.25e+05
idade_em_anos     -1.286e+04     66.533   -193.336      0.000    -1.3e+04   -1.27e+04
==============================================================================
Omnibus:                   407267.386   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        541968514.119
Skew:                          12.522   Prob(JB):                         0.00
Kurtosis:                     238.341   Cond. No.                     8.94e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.94e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""